In [1]:
import numpy as np

index = np.load("/home/jovyan/Sample_Based_Extension/UNSW/best_defense_indices.npy", allow_pickle=True)
index

array([['baseline', 0, 'Def1-11 Combined', 0.9923313825446824,
        0.9911800925969834, array([9, 3, 9, ..., 9, 9, 9])],
       ['BIM', 0.01, 'Def1-11 Combined', 0.9891551359556379,
        0.987484040917074, array([9, 3, 9, ..., 9, 9, 9])],
       ['BIM', 0.1, 'Def1-11 Combined', 0.9746251239350361,
        0.9705688736459472, array([1, 6, 1, ..., 1, 1, 1])],
       ['BIM', 0.2, 'Def1-11 Combined', 0.9594545980995165,
        0.9538946564599001, array([6, 6, 6, ..., 6, 6, 6])],
       ['BIM', 0.3, 'Def1-11 Combined', 0.919233840187416,
        0.9113183206844415, array([6, 6, 6, ..., 6, 6, 6])],
       ['FGSM', 0.01, 'Def1-11 Combined', 0.9889357819094331,
        0.9872271983425641, array([9, 3, 9, ..., 9, 9, 9])],
       ['FGSM', 0.1, 'Def1-11 Combined', 0.9680708250344385,
        0.9628361221532343, array([1, 6, 1, ..., 1, 1, 1])],
       ['FGSM', 0.2, 'Def1-11 Combined', 0.9461968395469023,
        0.9384213027985543, array([1, 6, 1, ..., 1, 1, 1])],
       ['FGSM', 0.3, 'Def1

In [2]:
attack_models = [
    ("BIM", [0.1]),
    ("FGSM", [0.1]),
    ("PGD", [0.1]),
    ("DF", [0.1]),
    ("AutoPGD", [0.1]),
    ("ZOO", [0.1]),
    ("CaFA", [0.1]),
    ("SINIFGSM", [0.1]),
    ("VNIFGSM", [0.1]),
]

all_adv_examples = []

for attack_name, epsilons in attack_models:
    for epsilon in epsilons:
        filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
        x_test_adv = np.load(filename)
        all_adv_examples.append(x_test_adv)

x_test_adv_0_1 = np.concatenate(all_adv_examples, axis=0)


In [3]:
y_labels = []

for row in index:
    attack_name, eps = row[0], row[1]
    if eps == 0.1:
        y_array = row[-1]
        y_labels.append(y_array)

y_label_0_1 = np.concatenate(y_labels, axis=0)

In [4]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(y_label_0_1)

y_label_0_1 = label_encoder.transform(y_label_0_1)

for index, class_label in enumerate(label_encoder.classes_):
    print(f"defense '{class_label}' encode {index}")

defense '1' encode 0
defense '2' encode 1
defense '3' encode 2
defense '4' encode 3
defense '5' encode 4
defense '6' encode 5
defense '7' encode 6
defense '8' encode 7
defense '9' encode 8
defense '10' encode 9
defense '11' encode 10


In [5]:
import numpy as np
from sklearn.cluster import KMeans

def stratified_cluster_split(X, y, test_size=0.5, random_state=42):
    X_sub_list = []
    y_sub_list = []
    X_rest_list = []
    y_rest_list = []

    classes = np.unique(y)

    rng = np.random.RandomState(seed=random_state)

    for c in classes:
        X_c = X[y == c]
        y_c = y[y == c]

        kmeans = KMeans(n_clusters=2, random_state=random_state)
        cluster_labels = kmeans.fit_predict(X_c)

        cluster_0_idx = np.where(cluster_labels == 0)[0]
        cluster_1_idx = np.where(cluster_labels == 1)[0]

        if len(cluster_0_idx) <= len(cluster_1_idx):
            base_idx = cluster_0_idx
        else:
            base_idx = cluster_1_idx

        n_select = int(len(X_c) * test_size)
        if n_select == 0:
            continue

        if len(base_idx) >= n_select:
            selected_idx = rng.choice(base_idx, size=n_select, replace=False)
        else:
            extra_needed = n_select - len(base_idx)
            other_idx = np.setdiff1d(np.arange(len(X_c)), base_idx)
            extra_idx = rng.choice(other_idx, size=extra_needed, replace=False)
            selected_idx = np.concatenate([base_idx, extra_idx])

        rest_idx = np.setdiff1d(np.arange(len(X_c)), selected_idx)

        X_sub_list.append(X_c[selected_idx])
        y_sub_list.append(y_c[selected_idx])
        X_rest_list.append(X_c[rest_idx])
        y_rest_list.append(y_c[rest_idx])

    X_sub = np.concatenate(X_sub_list, axis=0)
    y_sub = np.concatenate(y_sub_list, axis=0)
    X_rest = np.concatenate(X_rest_list, axis=0)
    y_rest = np.concatenate(y_rest_list, axis=0)

    return X_rest, X_sub, y_rest, y_sub

_, X_10, _, y_10 = stratified_cluster_split(x_test_adv_0_1, y_label_0_1, test_size=0.1, random_state=42)



In [6]:
X_10.shape

(205144, 56)

In [7]:
# from xgboost import XGBClassifier

# print('xgb')
# xgb10 = XGBClassifier()
# xgb10.fit(X_10, y_10)

print("RF")
from sklearn.ensemble import RandomForestClassifier
rf10 = RandomForestClassifier()
rf10.fit(X_10, y_10)

# print("DT")
# from sklearn.tree import DecisionTreeClassifier
# dt10 = DecisionTreeClassifier()
# dt10.fit(X_10, y_10)

xgb
RF
DT


DecisionTreeClassifier()

In [ ]:
import time
import numpy as np

y_test = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')

attack_models = [
    ("baseline", [0]),
    ("BIM", [0.01, 0.2, 0.3]),
    ("FGSM", [0.01, 0.2, 0.3]),
    ("PGD", [0.01, 0.2, 0.3]),
    ("DF", [0.01, 0.2, 0.3]),
    ("AutoPGD", [0.01, 0.2, 0.3]),
    ("ZOO", [0.01, 0.2, 0.3]),
    ("CaFA", [0.01, 0.2, 0.3]),
    ("SINIFGSM", [0.01, 0.2, 0.3]),
    ("VNIFGSM", [0.01, 0.2, 0.3]),
]

In [8]:


# base_path_template = "/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack}_eps_{eps}.npy"

# from sklearn.model_selection import train_test_split

# for attack_name, epsilons in attack_models:
#     start_time = time.time()
#     all_adv_examples = []
#     all_y = []
    
#     if attack_name != "baseline":
#         for epsilon in epsilons:
#             filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
#             x_test_adv = np.load(filename)
#             all_adv_examples.append(x_test_adv)
#             all_y.append(y_test)
    
#         X = np.concatenate(all_adv_examples, axis=0)
#         y = np.concatenate(all_y, axis=0)
#     else:
#         X = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_test.npy')
#         y = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')

#     y_pred = xgb10.predict(X)
#     print(y_pred.shape)
#     unique_classes, count = np.unique(y_pred, return_counts=True)
#     print(unique_classes, count)
#     for class_num in unique_classes:
#         indices = np.where(y_pred == class_num)[0]
#         X_class = X[indices]
#         y_class = y[indices]
#         print(X_class.shape, y_class.shape)
    
#         x_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/XGB_Cluster/UNSW_Input10/x_test_adv_{attack_name}_Def{class_num+1}.npy"
#         y_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/XGB_Cluster/UNSW_Input10/y_test_adv_{attack_name}_Def{class_num+1}.npy"
        
#         np.save(x_filename, X_class)
#         np.save(y_filename, y_class)
    
#         print(f"Save {class_num} to {attack_name}_Def{class_num+1}.npy")

#     end_time = time.time()
#     result = end_time - start_time
#     print(f"Execution Time: {result:.6f} seconds")
    

(113971,)
[ 0  1  2  3  4  5  6  7  8  9 10] [18625   509 21860    80  3688 21516     1   239 39206  2802  5445]
(18625, 56) (18625,)
Save 0 to baseline_Def1.npy
(509, 56) (509,)
Save 1 to baseline_Def2.npy
(21860, 56) (21860,)
Save 2 to baseline_Def3.npy
(80, 56) (80,)
Save 3 to baseline_Def4.npy
(3688, 56) (3688,)
Save 4 to baseline_Def5.npy
(21516, 56) (21516,)
Save 5 to baseline_Def6.npy
(1, 56) (1,)
Save 6 to baseline_Def7.npy
(239, 56) (239,)
Save 7 to baseline_Def8.npy
(39206, 56) (39206,)
Save 8 to baseline_Def9.npy
(2802, 56) (2802,)
Save 9 to baseline_Def10.npy
(5445, 56) (5445,)
Save 10 to baseline_Def11.npy
Execution Time: 0.360775 seconds
(341913,)
[ 0  1  2  3  4  5  6  7  8  9 10] [148724    530   8744    873    917 152717     37    572  28167    398
    234]
(148724, 56) (148724,)
Save 0 to BIM_Def1.npy
(530, 56) (530,)
Save 1 to BIM_Def2.npy
(8744, 56) (8744,)
Save 2 to BIM_Def3.npy
(873, 56) (873,)
Save 3 to BIM_Def4.npy
(917, 56) (917,)
Save 4 to BIM_Def5.npy
(152717

In [9]:
from sklearn.model_selection import train_test_split

for attack_name, epsilons in attack_models:
    start_time = time.time()
    all_adv_examples = []
    all_y = []
    
    if attack_name != "baseline":
        for epsilon in epsilons:
            filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
            x_test_adv = np.load(filename)
            all_adv_examples.append(x_test_adv)
            all_y.append(y_test)
    
        X = np.concatenate(all_adv_examples, axis=0)
        y = np.concatenate(all_y, axis=0)
    else:
        X = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_test.npy')
        y = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')

    y_pred = rf10.predict(X)
    print(y_pred.shape)
    unique_classes, count = np.unique(y_pred, return_counts=True)
    print(unique_classes, count)
    for class_num in unique_classes:
        indices = np.where(y_pred == class_num)[0]
        X_class = X[indices]
        y_class = y[indices]
        print(X_class.shape, y_class.shape)
    
        x_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF_Cluster/UNSW_Input10/x_test_adv_{attack_name}_Def{class_num+1}.npy"
        y_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/RF_Cluster/UNSW_Input10/y_test_adv_{attack_name}_Def{class_num+1}.npy"
        
        np.save(x_filename, X_class)
        np.save(y_filename, y_class)
    
        print(f"Save {class_num} to {attack_name}_Def{class_num+1}.npy")

    end_time = time.time()
    result = end_time - start_time
    print(f"Execution Time: {result:.6f} seconds")
    

(113971,)
[ 0  1  2  3  4  5  7  8  9 10] [15784  1011 32930    77  1026 15148   409 38223  8377   986]
(15784, 56) (15784,)
Save 0 to baseline_Def1.npy
(1011, 56) (1011,)
Save 1 to baseline_Def2.npy
(32930, 56) (32930,)
Save 2 to baseline_Def3.npy
(77, 56) (77,)
Save 3 to baseline_Def4.npy
(1026, 56) (1026,)
Save 4 to baseline_Def5.npy
(15148, 56) (15148,)
Save 5 to baseline_Def6.npy
(409, 56) (409,)
Save 7 to baseline_Def8.npy
(38223, 56) (38223,)
Save 8 to baseline_Def9.npy
(8377, 56) (8377,)
Save 9 to baseline_Def10.npy
(986, 56) (986,)
Save 10 to baseline_Def11.npy
Execution Time: 1.627028 seconds
(341913,)
[ 0  1  2  3  4  5  7  8  9 10] [154186    226  43625    989    207 120283     93  22250     48      6]
(154186, 56) (154186,)
Save 0 to BIM_Def1.npy
(226, 56) (226,)
Save 1 to BIM_Def2.npy
(43625, 56) (43625,)
Save 2 to BIM_Def3.npy
(989, 56) (989,)
Save 3 to BIM_Def4.npy
(207, 56) (207,)
Save 4 to BIM_Def5.npy
(120283, 56) (120283,)
Save 5 to BIM_Def6.npy
(93, 56) (93,)
Save 

In [10]:
# from sklearn.model_selection import train_test_split

# for attack_name, epsilons in attack_models:
#     start_time = time.time()
#     all_adv_examples = []
#     all_y = []
    
#     if attack_name != "baseline":
#         for epsilon in epsilons:
#             filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_{attack_name}_eps_{epsilon}.npy'
#             x_test_adv = np.load(filename)
#             all_adv_examples.append(x_test_adv)
#             all_y.append(y_test)
    
#         X = np.concatenate(all_adv_examples, axis=0)
#         y = np.concatenate(all_y, axis=0)
#     else:
#         X = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_test.npy')
#         y = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')

#     y_pred = dt10.predict(X)
#     print(y_pred.shape)
#     unique_classes, count = np.unique(y_pred, return_counts=True)
#     print(unique_classes, count)
#     for class_num in unique_classes:
#         indices = np.where(y_pred == class_num)[0]
#         X_class = X[indices]
#         y_class = y[indices]
#         print(X_class.shape, y_class.shape)
    
#         x_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/DT_Cluster/UNSW_Input10/x_test_adv_{attack_name}_Def{class_num+1}.npy"
#         y_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/DT_Cluster/UNSW_Input10/y_test_adv_{attack_name}_Def{class_num+1}.npy"
        
#         np.save(x_filename, X_class)
#         np.save(y_filename, y_class)
    
#         print(f"Save {class_num} to {attack_name}_Def{class_num+1}.npy")

#     end_time = time.time()
#     result = end_time - start_time
#     print(f"Execution Time: {result:.6f} seconds")
    

(113971,)
[ 0  1  2  3  4  5  7  8  9 10] [21361  1919 28387   163  3340 15503   707 37168  3377  2046]
(21361, 56) (21361,)
Save 0 to baseline_Def1.npy
(1919, 56) (1919,)
Save 1 to baseline_Def2.npy
(28387, 56) (28387,)
Save 2 to baseline_Def3.npy
(163, 56) (163,)
Save 3 to baseline_Def4.npy
(3340, 56) (3340,)
Save 4 to baseline_Def5.npy
(15503, 56) (15503,)
Save 5 to baseline_Def6.npy
(707, 56) (707,)
Save 7 to baseline_Def8.npy
(37168, 56) (37168,)
Save 8 to baseline_Def9.npy
(3377, 56) (3377,)
Save 9 to baseline_Def10.npy
(2046, 56) (2046,)
Save 10 to baseline_Def11.npy
Execution Time: 0.137421 seconds
(341913,)
[ 0  1  2  3  4  5  6  7  8  9 10] [118775   7089  20527   4861   2671 134161   3235  17196  19920  10452
   3026]
(118775, 56) (118775,)
Save 0 to BIM_Def1.npy
(7089, 56) (7089,)
Save 1 to BIM_Def2.npy
(20527, 56) (20527,)
Save 2 to BIM_Def3.npy
(4861, 56) (4861,)
Save 3 to BIM_Def4.npy
(2671, 56) (2671,)
Save 4 to BIM_Def5.npy
(134161, 56) (134161,)
Save 5 to BIM_Def6.npy

In [11]:
import numpy as np


attack_names = [
    "baseline", 
    "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

all_attack_list = []

model_name_list = ["XGB", "RF", "DT"]

for m_name in model_name_list:
    for attack_name in attack_names:
        each = []
        num = 0
        each.append(f"{m_name}10")
        for i in range(1, 12):
            y_filename = f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data/{m_name}_Cluster/UNSW_Input10/x_test_adv_{attack_name}_Def{i}.npy"
            try:
                y = np.load(y_filename)
                num+=y.shape[0]
                each.append(y.shape[0])
            except FileNotFoundError:
                each.append(0)
        each.append(num)
        all_attack_list.append(each)


In [12]:
import pandas as pd
all_attack_list = pd.DataFrame(all_attack_list)
all_attack_list.to_csv("./Dynamite_Num_Attack_Input10_Cluster.csv")

In [13]:
all_attack_list

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,XGB20,18625,509,21860,80,3688,21516,1,239,39206,2802,5445,113971
1,XGB20,148724,530,8744,873,917,152717,37,572,28167,398,234,341913
2,XGB20,145156,505,8658,1035,739,149605,33,5323,26424,4138,297,341913
3,XGB20,148724,530,8744,873,917,152717,37,572,28167,398,234,341913
4,XGB20,158424,29,1919,1619,278,135588,712,28127,2280,12376,561,341913
5,XGB20,149884,687,7686,1245,1559,151073,18,594,26138,822,2207,341913
6,XGB20,51668,1562,65940,236,11038,64077,6,717,121976,8378,16315,341913
7,XGB20,153300,264,38822,12,5713,79643,2,696,27016,10586,25859,341913
8,XGB20,142865,1226,9650,2147,695,150915,2,336,32647,1330,100,341913
9,XGB20,146653,794,8770,2090,841,150613,3,615,30469,720,345,341913
